In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
data = pd.read_csv('../data/data.csv')

In [3]:
#Keep only actual sales and remove uneeded cols
def drop_cols_and_rows(data, cols_to_drop, row_to_keep):
    data = data[data['FieldID'].isin(row_to_keep)]
    data.drop(columns=cols_to_drop, axis=1, inplace=True)
    return data


#Rename cols
def col_renamer(data, rename_dict):
    for col_name in list(rename_dict.keys()):
        data.rename(columns={col_name:rename_dict[col_name]}, inplace=True)
    return data


#Select cols to keep
def col_selecter(data, selected_cols):
    data = data[selected_cols]
    return data


#Make artificial time index to pass into dataframe builer
def create_time_index(data):
    data['Date']= pd.to_datetime(data['Date'])
    data["Date"].dt.year * 12
    data["time_idx"] = data["Date"].dt.year * 12 + data["Date"].dt.month
    data["time_idx"] -= data["time_idx"].min()
    data["month"] = data.Date.dt.month.astype(str).astype("category")
    return data


#Create average cols for groups, remove volume under zero
def feature_engineering(data):
    data.Volume=data.Volume.mask(data.Volume.lt(0),0)
    data["log_volume"] = np.log(data.Volume + 1e-8)
    data["avg_volume_by_material"] = data.groupby(["time_idx", "ItemID"], observed=True).Volume.transform("mean")
    data["avg_volume_by_plant"] = data.groupby(["time_idx", "PlantID"], observed=True).Volume.transform("mean")
    data = data.assign(
                        timeseries=pd.factorize(list(zip(
                        *[data[c].values.tolist() for c in ['PlantID', 'ItemID']]
                        )))[0] + 1
                        )
    return data


#Only use id want to remove final time index
def remove_last_time_idx(data):
    data = data[data['time_idx'] != 35]
    return data


#Print summary of data
def data_summary(data):
    print('')
    print('Data size:')
    print(data.shape)
    print('Data columns:')
    print(data.columns.values)
    print('')


#Link all functions together
def preprocessing(data, cols_to_drop, row_to_keep, rename_dict, selected_cols):
    data_summary(data)
    data = drop_cols_and_rows(data, cols_to_drop, row_to_keep)
    data = col_renamer(data, rename_dict)
    data = col_selecter(data, selected_cols)
    data = create_time_index(data)
    data = feature_engineering(data)
    data = remove_last_time_idx(data)
    data_summary(data)
    return data

In [6]:
#Run function for preprocessing
COLS_TO_DROP = ['Field', 'UpdateBy', 'UpdateWhen', 'PlanningPeriod', 'ProductLineID', 'PeriodDiff']
ROW_TO_KEEP = ['FNLBSFRC']
RENAME_DICT = {'FieldContent':'Volume', 'PlannedPeriod':'Date'}
SELECTED_COLS = ['PlantID', 'ItemID', 'Volume', 'Date']
data_processed = preprocessing(data, COLS_TO_DROP, ROW_TO_KEEP, RENAME_DICT, SELECTED_COLS)


Data size:
(2961504, 11)
Data columns:
['ProductLineID' 'PlanningPeriod' 'ItemID' 'PlantID' 'PlannedPeriod'
 'PeriodDiff' 'FieldID' 'Field' 'FieldContent' 'UpdateBy' 'UpdateWhen']



C:\Users\Joseph.Amess\Anaconda3\lib\site-packages\pandas\core\frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(
C:\Users\Joseph.Amess\Anaconda3\lib\site-packages\pandas\core\frame.py:5039: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(



Data size:
(423072, 10)
Data columns:
['PlantID' 'ItemID' 'Volume' 'Date' 'time_idx' 'month' 'log_volume'
 'avg_volume_by_material' 'avg_volume_by_plant' 'timeseries']



In [7]:
pd.DataFrame.to_csv(data_processed, '../data/data_forecast.csv')